In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/blur-dataset/defocused_blurred/0_IPHONE-SE_F.JPG
/kaggle/input/blur-dataset/defocused_blurred/330_XIAOMI-PROCOFONE-F1_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/46_IPHONE-8-PLUS_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/121_HONOR-7X_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/337_XIAOMI-PROCOFONE-F1_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/43_HONOR-8X_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/319_XIAOMI-REDMI-NOTE-4X_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/123_NIKON-D3400-35MM_F.JPG
/kaggle/input/blur-dataset/defocused_blurred/306_XIAOMI-REDMI-5-PLUS_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/33_HONOR-8X_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/187_XIAOMI-REDMI-4A_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/132_NIKON-D3400-18-55MM_F.JPG
/kaggle/input/blur-dataset/defocused_blurred/317_IPHONE-8-PLUS_F.jpg
/kaggle/input/blur-dataset/defocused_blurred/341_XIAOMI-PROCOFONE-F1_F.jpg
/kaggle/input/bl

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
#import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

In [6]:
BATCH_SIZE = 64
IMG_SIZE = (160, 160)
directory = "/kaggle/input/pizza-not-pizza/pizza_not_pizza/"
blurred_dataset = image_dataset_from_directory("/kaggle/input/blur-dataset/defocused_blurred/",
                                             labels=None,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             seed=42)
clear_dataset = image_dataset_from_directory("/kaggle/input/blur-dataset/blur_dataset_scaled/sharp/",
                                             labels=None,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             seed=42)

Found 350 files belonging to 1 classes.


2022-08-19 16:50:44.214543: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 350 files belonging to 1 classes.


In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
blurred_dataset = blurred_dataset.prefetch(buffer_size=AUTOTUNE)
clear_dataset = clear_dataset.prefetch(buffer_size=AUTOTUNE)

In [33]:
input = tf.keras.layers.Input(shape=(160, 160, 3))

# Encoder
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)

# Decoder
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, activation="relu", padding="same")(x)
x = tf.keras.layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)



In [35]:
# Autoencoder
autoencoder = tf.keras.Model(input, x)
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")
autoencoder.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 160, 160, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 80, 80, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 80, 80, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 32)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 80, 80, 32)        9248      
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 160, 160, 32)      9248

In [38]:
autoencoder.fit(
    x=blurred_dataset,
    y=clear_dataset,
    epochs=5,
    batch_size=BATCH_SIZE,
    shuffle=True
)

ValueError: `y` argument is not supported when using dataset as input.